In [ ]:
import os
import glob
from PIL import Image
import matplotlib.pyplot as plt

base_dirs = [
    "/home/shared_ws6/data/unreal_engine/local_traverse_fov90",
    "/home/shared_ws6/data/unreal_engine/spirals",
]


def find_rover0_image(dataset_root):
    # Check if this IS rover_0 folder (case for spirals/rover_0)
    if os.path.basename(dataset_root) == "rover_0":
        rover_path = dataset_root
    else:
        # Check for rover_0 subdir (case for short_base/rover_0)
        rover_path = os.path.join(dataset_root, "rover_0")

    if not os.path.exists(rover_path):
        # Some folders in local_traverse_fov90 might be metadata or files, skip them silently or handle
        return None

    # Find camera folder
    # Prefer cam_front, then cam_front_left
    cam_dirs = glob.glob(os.path.join(rover_path, "cam_*"))
    cam_dir = None

    # Priority: cam_front -> cam_front_left
    for c in cam_dirs:
        if "cam_front" == os.path.basename(c):
            cam_dir = c
            break

    if not cam_dir:
        for c in cam_dirs:
            if "cam_front_left" == os.path.basename(c):
                cam_dir = c
                break

    # Fallback to any cam if specific ones not found
    if not cam_dir and cam_dirs:
        cam_dir = cam_dirs[0]

    if not cam_dir:
        return None

    rgb_dir = os.path.join(cam_dir, "rgb")
    if not os.path.exists(rgb_dir):
        return None

    images = sorted(glob.glob(os.path.join(rgb_dir, "*.png")))
    if not images:
        return None

    return images[0]


datasets = []

# local_traverse_fov90
p1 = base_dirs[0]
if os.path.exists(p1):
    for d in sorted(os.listdir(p1)):
        full_p = os.path.join(p1, d)
        if os.path.isdir(full_p):
            datasets.append(full_p)

# spirals
p2 = base_dirs[1]
# For spirals, user requested ONLY rover_0
rover0_spiral = os.path.join(p2, "rover_0")
if os.path.exists(rover0_spiral):
    datasets.append(rover0_spiral)

print(f"Found {len(datasets)} candidate dataset folders")

# Display
for ds_path in datasets:
    img_path = find_rover0_image(ds_path)
    if img_path:
        print(f"\nDataset: {os.path.basename(ds_path)}")
        print(f"Path: {img_path}")
        try:
            img = Image.open(img_path)
            plt.figure(figsize=(10, 6))
            plt.imshow(img)
            # plt.title(f"{os.path.basename(ds_path)} - rover_0")
            plt.axis("off")
            plt.show()
        except Exception as e:
            print(f"Error loading image: {e}")
    else:
        # Optional: Print skipped if expected
        pass